### This script generates energy models (ePWMs) from relative binding energy (RBE) matrix for each CRX HD in the Spec-seq experiment.

In [1]:
import os
import sys
import re
import glob2
import itertools
import subprocess

import numpy as np
import pandas as pd
from scipy import stats
from sklearn import linear_model

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager

In [ ]:
# get hex color code of matplotlib palette
from pylab import *

cmap = cm.get_cmap('tab20')  # matplotlib color palette name, n colors
for i in range(cmap.N):
    rgb = cmap(i)[:3]  # will return rgba, we take only first 3 so we get rgb
    print(matplotlib.colors.rgb2hex(rgb))

### 0. Housekeeping

In [2]:
# set the parent directory as the working directory
# compatible naming between WinOS and MacOS
base_dir = os.path.dirname(os.getcwd())
os.chdir(base_dir)
sys.path.insert(0, os.path.join(base_dir,"utils"))

In [ ]:
from utils import specseq_plot_utils, specseq_ewm_utils, calculate_ratios

In [3]:
# set up directory variables
# Specseq data
spec_basedir = os.path.join(base_dir, "Specseq_analysis", "processed_data")
scriptdir=os.path.join(base_dir, "Specseq_analysis")
# CRX ChIP data
chip_basedir = os.path.join(base_dir,"ChIPseq_analysis","processed_data")
# CRX RNA data
rna_basedir = os.path.join(base_dir,"RNAseq_analysis","processed_data")
# Aldiri RNA data
devrna_dir = os.path.join(base_dir,"RNAseq_analysis","processed_data")
# figure output directory
outbase_dir = os.path.join(base_dir, "Figures")

In [8]:
specseq_plot_utils.set_manuscript_params()
# Define library design (update as new libraries added)
lib_designs = {
    # for automatically adding filter sequences
    "M":"TAANNN",
    "Mrev":"NNNTTA",
}
# Define the two colormaps: classic and colorblind safe
dna_classic = {
    # as in most of the pwm figures
    "A": [0, 0.5, 0],
    "C": [0, 0, 1],
    "G": [1, 0.65, 0],
    "T": [1, 0, 0],
    "filler": "#979797"
}
dna_safe = {
    # from the Stormo lab
    "A": "#0E927B",
    "C": "#59A9D8",
    "G": "#DC9514",
    "T": "#1A1A1A",
    "filler": "#979797"
}

### I. Retrieve all RBE files in the given folder

In [9]:
# directories containing processed RBE data
RBE_dir=os.path.join(spec_basedir, "RBE","avgRBE")
figure_dir=os.path.join(base_dir, "Figures")
# retrieve RBE files with specified order
RBE_filelist = [os.path.join(RBE_dir, f"{p}_avgRBE.txt") for p in ["wt", "e80a", "k88n", "r90w"]]
# alternative: list all avg RBE file in directory without specifying order
#RBE_filelist=list(glob2.glob(os.path.join(RBE_dir,"*_avgRBE.txt")))

all_energy_matrix = {os.path.split(RBE_filelist[i])[1].split("_avgRBE.txt")[0]:pd.read_csv(RBE_filelist[i], sep="\t", header=0) for i in range(len(RBE_filelist))}
# convert to series
all_energy_matrix = pd.Series(all_energy_matrix)

### II. Iterate through all RBE data and build energy model for selected library and band in each dataset

In [10]:
all_energy_model = specseq_ewm_utils.ewm_from_RBE(all_energy_matrix, lib_list=["M", "Mrev"], find_band="m", normalize=False)
all_energy_model.index

wt.M.m
  - fitting energy data avg.M.mddG with linear model
wt.Mrev.m
  - fitting energy data avg.Mrev.mddG with linear model
e80a.M.m
  - fitting energy data avg.M.mddG with linear model
e80a.Mrev.m
  - fitting energy data avg.Mrev.mddG with linear model
k88n.M.m
  - fitting energy data avg.M.mddG with linear model
k88n.Mrev.m
  - fitting energy data avg.Mrev.mddG with linear model
r90w.M.m
  - fitting energy data avg.M.mddG with linear model
r90w.Mrev.m
  - fitting energy data avg.Mrev.mddG with linear model


Index(['wt.M.m', 'wt.Mrev.m', 'e80a.M.m', 'e80a.Mrev.m', 'k88n.M.m',
       'k88n.Mrev.m', 'r90w.M.m', 'r90w.Mrev.m'],
      dtype='object')

In [ ]:
# check one ePWM to make sure the formatting is correct
all_energy_model["wt.M.m"]

In [11]:
# Compile and save all energy models in the dictionary into a single file
specseq_ewm_utils.save_ewm(all_energy_model,os.path.join(spec_basedir,"specseq_ewm_sklearn.txt"))

### II.alternative: Generate energy models with lm() from specseq_mlr.R 
#### written as described in the original paper, gives exact same result as Python scikit-learn LinearRegression but slower

In [ ]:
all_mlr_energy_model = specseq_ewm_utils.mlr_ewm_from_RBE(scriptdir, RBE_filelist, lib_list=["M","Mrev"], find_band="m")
print(all_mlr_energy_model.index)

In [ ]:
specseq_ewm_utils.save_ewm(all_mlr_energy_model,os.path.join(spec_basedir,"specseq_ewm_mlr.txt"))

#### Comparison of energy matrix from sklearn (python) and from lm (R)

In [ ]:
# lr model from Python scikit-learn LinearRegression function (if row min = 0kT, the ePWM is not normalized)
all_energy_model["wt.M.m"]

In [ ]:
# lr model from R lm() function (ePWMs are automatically normalized, i.e. row sum = 0kT)
all_mlr_energy_model["wt.M.m"]

### III. if energy models already generated, read directly from file

In [ ]:
all_energy_model = specseq_ewm_utils.read_ewm_files(os.path.join(spec_basedir,"specseq_ewm_sklearn.txt"))

### IV. Compare predicted with observed relative binding energy

In [ ]:
# predict energy for all sequences based on ePWM
predicted_energy_ser = {}
for name in all_energy_model.index:
    p = name.split(".")[0]
    lib = name.split(".")[1]
    band = name.split(".")[2]
    # need to normalize to make row sum 0 if not already done
    ewm = specseq_ewm_utils.normalize_ewm(all_energy_model[name].copy())
    # predict relative binding energy based on ewm
    predicted_energy_df = specseq_ewm_utils.predict_bindingEnergy(lib_designs[lib], ewm)
    predicted_energy_ser[name] = predicted_energy_df
predicted_energy_ser = pd.Series(predicted_energy_ser)

In [ ]:
def plot_obs_vs_pred_energy(observed_energy_df, predicted_energy_df, identifier=None, stepsize=0.4, figax=None):
    # compile observed and predicted energy data
    # infer reference sequence from observed data (by definition ddG = 0)
    observed_energy_df = observed_energy_df.set_index("sequence")
    obs_energy_col = [x for x in observed_energy_df.columns.tolist() if re.search("ddG", x)]
    # infer energy column to plot from identifier if specified, else plot the first energy column
    identifier = "avg."+identifier+"ddG"
    print(identifier)
    if identifier in obs_energy_col:
        obs_energy_col = identifier
    else :
        obs_energy_col = obs_energy_col[0]
    ref_seq = observed_energy_df[observed_energy_df[obs_energy_col] == 0].index[0]
    print("reference seqeunce identified: " + ref_seq)

    # offset the predicted energy by energy of the infered reference sequence
    predicted_energy_df[["pred.ddG"]] = predicted_energy_df[["pred.ddG"]] - predicted_energy_df.loc[ref_seq,"pred.ddG"]

    # Set up the figure
    if figax:
        fig, ax = figax
    else:
        fig, ax = plt.subplots()
    # compiled dataframes
    compiled_df = pd.merge(observed_energy_df[[obs_energy_col]], predicted_energy_df, left_index=True, right_index=True)
    fig, ax, text = specseq_plot_utils.make_correlation_scatter(compiled_df, [obs_energy_col,"pred.ddG"], colors="black",
                                            xticks=None, yticks=None, stepsize=stepsize, annotate_list=None, figname=None, figax=(fig,ax))
    
    # rotate x tick labels for spacing purpose
    specseq_plot_utils.rotate_ticks(ax.get_xticklabels(), 90)
    # add stats
    left,right = ax.get_xlim()
    bottom, top = ax.get_ylim()
    if bottom > -0.4:
        bottom = 0
    
    # format pearson's r
    r_value = text[0]
    ax.text(right-0.1, bottom+0.05, r_value, horizontalalignment='right', verticalalignment='bottom', style = 'italic')
    # add plot title
    #ax.set_title(protein_pair[0].upper()+" x "+protein_pair[1].upper()+"  "+lib+" lib")
    #ax.set_title("Relative Binding Energy [kT]", fontsize=12)
   
    # add plot title and axis titles
    ax.set_xlabel("obs.ddG")
    ax.set_ylabel("pred.ddG")
    ax.set_title(identifier+"\nobs vs. pred")

    return fig, ax

In [ ]:
# make obs vs pred ddG plots for selected protein
nplots = [n for n in all_energy_model.index if n.__contains__("wt")]
fig, axes = plt.subplots(ncols=len(nplots), nrows=1, figsize=(4,2), dpi=150,
                            constrained_layout=True)
axes = axes.flatten()
for i,n in enumerate(nplots):
    # infer protien, library and band from energy model id
    print(f"ploting {n}")
    p = n.split(".")[0]
    lib = n.split(".")[1]
    band = n.split(".")[2]
    # retreive predicted energy dataframe
    predicted_energy_df = predicted_energy_ser[n].copy()
    # plot observed vs predicted relative binding energy
    mask = lambda x: x.MMcount <= 2
    fig, axes[i] = plot_obs_vs_pred_energy(all_energy_matrix[p].loc[mask,:], predicted_energy_df, identifier=lib+"."+band, stepsize=0.4, figax=(fig, axes[i]))

### V. Explore relative ranking of motif variants

In [ ]:
def motif_rank_order_dots(energy_df, annotate=None, y_column="pred.ddG", ticklabel_size=3, markersize=1, title=None, colors=["#1A1A1A", "#DC9514"], figax=None):
    if figax:
        fig, ax = figax
    else:
        fig, ax = plt.subplots()

    energy_df = energy_df.copy()
    # initilize an array for y axis
    y_pos = np.arange(len(energy_df.index))

    # add selected sequences on ticks
    ax.set_yticks(y_pos)
    labels = [item.get_text() for item in ax.get_yticklabels()]
    for i,seq in enumerate(energy_df.sequence):
        if annotate: #highlight those interesting ones
            if seq in annotate:
                ax.scatter(y=i,x=energy_df.at[i,y_column], color=colors[1], marker="o", s=markersize)
            else:
                ax.scatter(y=i,x=energy_df.at[i,y_column], color=colors[0], marker="o", s=markersize) 
        else:
            ax.scatter(y=i,x=energy_df.at[i,y_column], color=colors[0], marker="o", s=markersize)
        labels[i] = seq

    ax.yaxis.set_ticks_position("right")
    ax.set_yticklabels(labels, fontsize=ticklabel_size, va="center", ha="left")
    ax.tick_params(axis='y', which='major', pad=2)

    # add color to the labels
    for i,seq in enumerate(energy_df.sequence):
        if annotate: #highlight those interesting ones
            if seq in annotate:
                ax.get_yticklabels()[i].set_color(colors[1])

    # add axis decorations
    ax.set_xlabel("$\Delta \Delta G$ (kcal/mol)")

    if title:
        ax.set_title(title)

    return fig, ax

In [ ]:
# setup figure
fig, ax_list = plt.subplots(nrows=1, ncols=4, figsize=(6,3),
                            dpi=300, constrained_layout=True,
                            gridspec_kw=dict(wspace=0.1))

# make ranked motif dotplot
lib = "M"

annotation_library = {  "M":["TAATTA", "TAATTT", "TAATCC", "TAAGCC"],
                        "Mrev": ["TAATTA", "AAATTA", "GGATTA", "GGCTTA"]}

for i,(name,ref) in enumerate(zip([f"wt.{lib}.m", f"r90w.{lib}.m", f"e80a.{lib}.m", f"k88n.{lib}.m"], ["TAATCC"]*3+["TAATTA"])): 
    #use library design instead of consensus sequence if want to plot all sequences in the library
    predicted_energy_df = predicted_energy_ser[name].copy()
    # annotate mismatches and keep only MMcount<=2
    # because the predicted dataframe has not library column but the count_mistmatches function requires that
    predicted_energy_df["lib"] = lib
    predicted_energy_df = calculate_ratios.count_mismatches(predicted_energy_df, lib, ref)
    mask = lambda x: x.MMcount <= 2
    predicted_energy_df = predicted_energy_df.loc[mask,:]
    # sort by energy consensus (bottom) to weakest (top)
    predicted_energy_df = predicted_energy_df.sort_values(by="pred.ddG", ascending=False).reset_index(drop=False)

    # normalize to consensus
    predicted_energy_df["pred.ddG"] = predicted_energy_df["pred.ddG"] - predicted_energy_df.loc[lambda df: df.sequence == ref, "pred.ddG"].item()

    ax = ax_list[i]
    fig, ax = motif_rank_order_dots(predicted_energy_df, annotate=annotation_library[lib], ticklabel_size=3, title=name, colors=["#1A1A1A", "#DC9514"], figax=(fig, ax))

for i in range(4):
    ax_list[i].set_xlim(-.5,3.5)


In [ ]:
specseq_plot_utils.save_fig(fig, prefix=os.path.join(figure_dir, "ewm_rank_order", f"{lib}_library"), timestamp=False, tight_layout=False)